In [1]:
import os
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from utils.utils import pretty_lat, pretty_lon
from tempfile import TemporaryDirectory

In [2]:
from station.data_submission import DataSubmission
from station.station import StationData

from train_station_twin.training_executer import TrainingExecuter
from infilling.evaluation_executer import EvaluatuionExecuter
from infilling.infilling_writer import InfillingWriter

from era5.era5_for_station import DownloadEra5ForStation, DownloadEra5ForStationGaps
from era5.era5_download_hook import Era5DownloadHook


In [3]:
def plot_n_steps_of_area_from_nc_file(path, n=1, vars="tas", title="", vmin=None, vmax=None):

    dataset = xr.open_dataset(path)

    n = min(n, dataset.time.size)
    time_index_list = np.random.choice(dataset.time.size, n, replace=False)

    lat_slice = slice(None)
    lon_slice = slice(None)
    _lon = dataset.lon.values[lon_slice]
    _lat = dataset.lat.values[lat_slice]

    # if not list make it a list
    if not isinstance(vars, list):
        vars = [vars]

    for time_index in time_index_list:
        # set title
        title += f"\n{dataset.time.values[time_index].astype('datetime64[s]').astype('O')}"

        # subtitle lat lon area
        subtitle = f"\nLat: {pretty_lat(_lat[0])} to {pretty_lat(_lat[-1])}" + \
            f"\nLon: {pretty_lon(_lon[0])} to {pretty_lon(_lon[-1])}"

        for var in vars:

            # plot
            fig, ax = plt.subplots(
                subplot_kw={'projection': ccrs.PlateCarree()})
            # Plot the temperature data with a quadratic colormap
            _data = dataset.variables[var].values[time_index, lat_slice, lon_slice]
            pcm = ax.pcolormesh(_lon, _lat, _data, cmap='viridis',
                                shading='auto', vmin=vmin, vmax=vmax)

            # Add coastlines
            ax.coastlines()

            # Add colorbar
            cbar = plt.colorbar(pcm, ax=ax, label='Temperature')

            # Set labels and title
            ax.set_xlabel('Longitude')
            ax.set_ylabel('Latitude')
            plt.title(title + (f"\n[{var}]" if len(vars) > 1 else ""))

            # position title a higher
            plt.subplots_adjust(top=1)
            
            # Add subtitle
            plt.figtext(0.125, 0.05, subtitle, wrap=True, horizontalalignment='left', fontsize=12)

            # Show the plot
            plt.show()

In [4]:
marshall_data = DataSubmission("Marshall_2017-2023")
marshall_data.measurement_dir_path = "./measurements/Marshall/"

marshall_station = StationData(
    name=marshall_data.name,
    folder_path=marshall_data.measurement_dir_path
)

Extracting Marshall_2017-2023...


  0%|          | 0/2465 [00:00<?, ?it/s]

100%|██████████| 2465/2465 [02:28<00:00, 16.55it/s]


In [5]:

training = TrainingExecuter(station=marshall_station)

output_model_path = training.execute()
marshall_data.add_model(output_model_path)



(43635,)
Saving to /var/folders/yj/k7pg73d56rl366r8bvlbncgm0000gn/T/tmp9bil852x/train/marshall_2017-2023.nc


2024-04-15 21:15:36,689 INFO Welcome to the CDS
2024-04-15 21:15:36,690 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-04-15 21:15:36,821 INFO Request is queued
2024-04-15 21:15:37,902 INFO Request is failed
2024-04-15 21:15:37,905 ERROR Message: the request you have submitted is not valid
2024-04-15 21:15:37,906 ERROR Reason:  Request too large. Requesting 61344 items, limit is 60000
2024-04-15 21:15:37,911 ERROR   Traceback (most recent call last):
2024-04-15 21:15:37,920 ERROR     File "/opt/cds/cdsinf/python/lib/cdsinf/runner/dispatcher.py", line 163, in _consume
2024-04-15 21:15:37,926 ERROR       result = handle_locally()
2024-04-15 21:15:37,928 ERROR     File "/opt/cds/cdsinf/python/lib/cdsinf/runner/dispatcher.py", line 252, in <lambda>
2024-04-15 21:15:37,929 ERROR       lambda: self.handle_exception(context, e),
2024-04-15 21:15:37,930 ERROR     File "/opt/cds/cdsinf/python/lib/cdsinf/runner/dispatcher.py", line 3

Exception: the request you have submitted is not valid. Request too large. Requesting 61344 items, limit is 60000.

In [ ]:

training.visualize()
training.cleanup()

NameError: name 'training' is not defined

In [ ]:
barbados_station = StationData("Barbados", "./measurements/Barbados/")
vienna_station = StationData("Vienna", "./measurements/Vienna/")

Extracting Barbados...


100%|██████████| 1263/1263 [01:26<00:00, 14.64it/s]


Extracting Vienna...


100%|██████████| 1857/1857 [02:03<00:00, 15.03it/s]
